In [1]:
#code de la demo 3
function pivot!(M::Matrix, i::Int, j::Int)
    m, n = size(M)
    @assert M[i, j] != 0
    M[i, :] = M[i, :]/M[i, j]
    for k in setdiff(1:m, i)
        M[k, :] -= M[k, j] * M[i, :]
    end
    return M
end

function getReducedCosts(M::Matrix)
    m, n = size(M)
    return M[end, 1:n-1]
end
function getxB(M::Matrix)
    m, n = size(M)
    return M[1:m-1, end]
end
function enteringVar(M::Matrix)
    rc = getReducedCosts(M)
    index = argmin(rc)
    return rc[index] >= 0 ? -1 : index
end
function exitingVarIndex(M::Matrix{T}, enteringVar::Int) where T
    col = M[1:end-1, enteringVar]
    xB = getxB(M)
    m, n = size(M)
    index = -1
    val = T(Inf)
    for i in 1:m-1
        if (col[i] > 0) && (xB[i]/col[i] < val)
            val = xB[i]/col[i]
            index = i
        end
    end
    return index
end
function isOneHot(v::Vector)
    n = length(v)
    return (sum(iszero, v) == n-1) && (sum(isone, v) == 1)
end 
function isoptimal(M)
    return enteringVar(M) == -1
end
function findInitialBasis!(M::Matrix)
    m, n = size(M)
    m-=1
    n-=1
    basis = [-1 for _ in 1:m]
    for i in 1:n
        if isOneHot(M[1:end-1, i])
            index = findfirst(isone, M[:, i])
            basis[index] = i
        end
    end
    @assert !any(t-> t == -1, basis) "problem not caconical"
    for i in 1:m
        j = basis[i]
        pivot!(M, i, j)
    end
    return basis
end
function simplexSolver(A::Matrix{T}, b::Vector, c::Vector; 
        verbose::Bool = false) where T
    M = [A b; c' 0]
    basis = findInitialBasis!(M)
    k = 1
    nmax = 1000
    while !isoptimal(M) && k < nmax
        k+=1
        verbose && display(M)
        entering = enteringVar(M)
        entering == -1 && (println("------------"); return T[-1, -1])
        leaving = exitingVarIndex(M, entering)
        leaving == -1 && (println("------------"); return T[-1, -1])
        verbose && @show (entering, leaving)
        basis[leaving] = entering
        pivot!(M, leaving, entering)
    end
    verbose && display(M)
    m, n = size(M)
    xstar = zeros(T, n - 1)
    xstar[basis] = getxB(M)
    #return xstar #code modifié ici. On retourne le tableau du simplex au lieux de la solution
    return M
end

simplexSolver (generic function with 1 method)

In [2]:
#feasible vérifie si le tableau du simplexe contient une solution primale réalisable
function feasible(M::Matrix{T}) where T
    b = M[1:end-1, end]
    # Vérifier si tous les éléments de b sont non-négatifs
    return all(b .>= 0)
end

#retourrne la ligne du tableau du simplex dont la variable de base va sortir de la base
function findLeavingVarInDual(M::Matrix{T}) where T
    min_ratio = 0
    index_of_min_ratio = -1
    
    for i in 1:(size(M, 1) - 1)
        if M[i, end] < 0
            ratio = M[i, end] / M[i, i]
            if ratio < min_ratio
                min_ratio = ratio
                index_of_min_ratio = i
            end
        end
    end
    
    return index_of_min_ratio
end

#retourne la variable qui va entrer dans la base. Si aucune variable ne peut entrer dans
#la base, on retourne -1.
function findEnteringVarInDual(M::Matrix{T}, leaving::Int) where T
    n = size(M, 2) - 1  
    max_ratio = -Inf
    entering = -1
    for j in 1:n
        if M[leaving, j] < 0  
            cost_reduced = M[end, j]
            ratio = cost_reduced / M[leaving, j]
            if ratio > max_ratio  
                max_ratio = ratio
                entering = j
            end
        end
    end
    return entering
end

function DualsimplexSolver(A::Matrix{T}, b::Vector, c::Vector; 
        verbose::Bool = false) where T
    @assert all(c .>= 0) #dual feasibility
    M = [A b; c' 0]
    basis = findInitialBasis!(M)
    k = 1
    nmax = 1000
    while !feasible(M) && k < nmax
        k+=1
        verbose && display(M)
        leaving = findLeavingVarInDual(M)
        entering = findEnteringVarInDual(M, leaving)
        if entering == -1
            println("aucune variable ne peut entrer dans la base.")
            println("problème non réalisable")
        end
        verbose && @show (entering, leaving)
        basis[leaving] = entering
        pivot!(M, leaving, entering)
    end
    verbose && display(M)
    m, n = size(M)
    xstar = zeros(T, n - 1)
    xstar[basis] = getxB(M)
    return xstar
end

DualsimplexSolver (generic function with 1 method)

In [3]:
using LinearAlgebra

In [4]:
Atilde = [1//1 0 1 0; 0 1 2 1; 0 0 0 1]
A = -[Atilde -I(3)]
@show size(A)
b = -[3, 4, 2]
@show size(b)
c = [0, 3, 4, 5, 0, 0, 0]
@show size(c);

size(A) = (3, 7)
size(b) = (3,)
size(c) = (7,)


In [5]:
xstar = DualsimplexSolver(A, b, c, verbose=true);

4×8 Matrix{Rational{Int64}}:
 -1//1   0//1  -1//1   0//1  1//1  0//1  0//1  -3//1
  0//1  -1//1  -2//1  -1//1  0//1  1//1  0//1  -4//1
  0//1   0//1   0//1  -1//1  0//1  0//1  1//1  -2//1
  0//1   3//1   4//1   5//1  0//1  0//1  0//1   0//1

(entering, leaving) = (4, 3)


4×8 Matrix{Rational{Int64}}:
 -1//1   0//1  -1//1  0//1  1//1  0//1   0//1   -3//1
  0//1  -1//1  -2//1  0//1  0//1  1//1  -1//1   -2//1
  0//1   0//1   0//1  1//1  0//1  0//1  -1//1    2//1
  0//1   3//1   4//1  0//1  0//1  0//1   5//1  -10//1

LoadError: BoundsError: attempt to access 4×8 Matrix{Rational{Int64}} at index [-1, 1]